In [ ]:
from deep_orderbook.replayer import Replayer
from deep_orderbook.shaper import BookShaper
import numpy as np
from pylab import rcParams

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
rcParams['figure.figsize'] = 20, 4

In [ ]:
MARKETS = ["ETH-BTC", "BTC-USD", "ETH-USD"]

In [ ]:
# itertools for asyncio !
import asyncio
from aioitertools import iter, next, map, zip
from aioitertools import enumerate

In [ ]:
shaper = BookShaper()
file_replayer = Replayer('../crypto-trading/data/L2', date_regexp='2020')
areplay = file_replayer.replayL2_async(pair='ETHBTC', shaper=shaper)
await next(areplay)
batptr = await next(areplay)

In [ ]:
for i in range(100):
    batptr = await next(areplay)
print(f"bids:\n{batptr['bids'].head()}")
print(f"asks:\n{batptr['asks'].head()}")
print(f"prices:\n{batptr['price']}")
print(f"trades:\n{batptr['trades']}")

In [ ]:
multi_replay = file_replayer.multireplayL2_async(MARKETS)
d = await next(multi_replay)
d

In [ ]:
async def run(maxpoints=10):
    async for i in iter(range(maxpoints)):
        d = await next(multi_replay)
    #return d

task = asyncio.create_task(run())
await task

In [ ]:
genarr = shaper.gen_array_async(market_replay=multi_replay, markets=MARKETS)
_ = await next(genarr)
_

In [ ]:
genacc = shaper.accumulate_array(genarr, markets=MARKETS)
_ = await next(genacc)
_

In [ ]:
async for toshow in BookShaper.images(accumulated_arrays=genacc, every=1, LENGTH=512):
    plt.figure(figsize=(15,11))
    plt.imshow(toshow, origin="upper")
    clear_output(wait=True)
    plt.show()

In [ ]:
every = 2
LENGTH = 256
x = []
async for n,sec in enumerate(genacc):
    allim = []
    for symb, data in sec.items():
        arr = np.stack(data['bs'][-LENGTH:])
        im = arr
        im[:,:,0] /= 10
        im += 0.5
        allim.append(im)
    allim = np.concatenate(allim, axis=1)
    if n % every == 0:
        plt.figure(figsize=(15,11))
        toshow = allim.transpose(1,0,2)
        toshow = np.clip(toshow, 0, 1)
        plt.imshow(toshow, origin="lower")
        plt.show()
        # pr = np.stack(data['ps'])[-LENGTH:, :-1]
        # plt.plot(pr)
        # plt.show()
#        print(symb)
        clear_output(wait=True)
#        plt.show()
#    if n == 100:
#        break
    pass